In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score

from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from read_emg import *
from build_CNN import *

plt.style.use('seaborn-bright')
plt.rcParams['xtick.labelsize'] = 7.5
plt.rcParams['ytick.labelsize'] = 7.5

In [2]:
# %%time
# df = read_file()
# df.to_pickle('DataCollection.pkl')

In [2]:
df = pd.read_pickle('DataCollection.pkl')
Gesture_list = ['Cylinder Grasp', 'Wrist Extension', 'Fist', 'Finger mass extension', 'Opposition', 'Lateral Pinch']
df['Trial_num'] = df['Trial_num'].astype(int)

def set_custom_palette(series, max_color = 'red', other_color = 'orange'):
    max_val = series.max()
    pal = []
    
    for item in series:
        if item == max_val:
            pal.append(max_color)
        else:
            pal.append(other_color)
    return pal

### Calculating the similarity using the channel means
def cross_correlation_vector(A, B):
    A_vector = []
    B_vector = []
    for channel in range(0, A.shape[1]):
        A_temp = A[:, channel] 
        A_channel_mean = A_temp.mean()
        A_vector.append(A_channel_mean)
        
        B_temp = B[:, channel]
        B_channel_mean = B_temp.mean()
        B_vector.append(B_channel_mean)
        
    A_vector = np.array(A_vector)
    B_vector = np.array(B_vector)
    C = np.multiply(A_vector, B_vector).sum() / (np.sqrt((A_vector**2).sum() * (B_vector**2).sum()))

    return C

In [3]:
def CC_to_shift(df, subject_reference, subject_to_rotate, calibration_gesture_list, session_reference, session_to_rotate, ax = None):
#     print(f'Current Subject: {subject_to_rotate}')
    shift_range = range(0, 8)

    max_CC = 0
    final_status = False
    final_shift = 0

    ## Reference session will not get flipped, but the test session
    CC_list = []
    for flipped in [False, True]:

        for shift in shift_range:
            A_vector_all_gesture = np.zeros(shape = (1, len(calibration_gesture_list) * 8))
            B_vector_all_gesture = np.zeros(shape = (1, len(calibration_gesture_list) * 8))
#             print(f'Shift: {shift}')

            for calibration_gesture in calibration_gesture_list:
                df_A = df[(df['ID'] == subject_reference) & 
                          (df['Gesture'] == calibration_gesture) & 
                          (df['Trial_num'] <= 3) & 
                          (df['session'] == session_reference)].iloc[:, :8]            
                df_A = df_A[df_A.index > 100]
                A_value = df_A.iloc[:, :8].rolling(1).mean().dropna().values

                ## concatenation of all gestures for sessionA
                ## Check if it is all zeros, if so, replace that with the first gesture
                ## If not, concatenate the 2nd gesture with the first one and so on
                if not np.any(A_vector_all_gesture):
                    A_vector_all_gesture = A_value
                else:
                    length = np.minimum(len(A_vector_all_gesture), len(A_value))
                    A_vector_all_gesture = np.concatenate((A_vector_all_gesture[:length], A_value[:length]), axis = 1)

                if flipped == False:    

                    df_B = df[(df['ID'] == subject_to_rotate) & 
                              (df['Gesture'] == calibration_gesture) & 
                              (df['Trial_num'] <= 3) & 
                              (df['session'] == session_to_rotate)].iloc[:, :8]
                else:
    #                 print('Flipping the sensor\n')
                    df_B = flipping_sensor(df[(df['ID'] == subject_to_rotate)  & 
                                              (df['Gesture'] == calibration_gesture) & 
                                              (df['Trial_num'] <= 3) & 
                                              (df['session'] == session_to_rotate)]).iloc[:, :8]

                df_B = df_B[df_B.index > 100]
                B = df_B.iloc[:, :8].rolling(1).mean().dropna()     

                B_value = pd.concat((B,B), axis = 1).iloc[:,  0 + shift:8 + shift].values

                ## concatenation of all gestures for sessionB (shift indiviually and concatenate)
                ## Check if it is all zeros, if so, replace that with the first gesture
                ## If not, concatenate the 2nd gesture with the first one and so on
                if not np.any(B_vector_all_gesture):
                    B_vector_all_gesture = B_value
                else:
                    length = np.minimum(len(B_vector_all_gesture), len(B_value))
                    B_vector_all_gesture = np.concatenate((B_vector_all_gesture[:length], B_value[:length]), axis = 1)

            C = cross_correlation_vector(A_vector_all_gesture, B_vector_all_gesture)
#             print(f'CC: {C:.4f}\n')
            CC_list.append(C)

    try:
        shift_max = np.where(CC_list == np.max(CC_list))[0][0]
        palette = set_custom_palette(pd.Series(CC_list))

        if shift_max < 8:
            final_status = False
            final_shift = shift_max
        else:
            final_status = True
            final_shift = shift_max - 8
            
        
#         ax.bar(x = ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7',
#                            'F_S0', 'F_S1', 'F_S2', 'F_S3', 'F_S4', 'F_S5', 'F_S6', 'F_S7'], 
#                        height = CC_list, alpha = 0.7, color = palette)
#         ax.set_title(subject)
#         ax.set_ylabel('Cross-correlation')
#         plt.setp(ax.get_xticklabels(), rotation=45)

    except Exception as e:
        print(e)

#     print(f'Flipping needed? {final_status}\n')
    return final_status, final_shift


In [4]:
def shift_to_rotation(df, shift, subject_reference, subject_to_rotate, session_reference, session_to_rotate, flipped = False):
    df_train = df[(df['ID'] == subject_reference) & 
                  (df['session'] == session_reference) & (df['Trial_num'] <=3) & 
                  (df['Gesture'] != '1') & (df['Gesture'] != '0') & (df['Gesture'] != '3') & 
                  (df['Gesture'] != '10') & (df['Gesture'] != '11')]
        
    df_test = df[(df['ID'] == subject_to_rotate) & 
                 (df['session'] == session_to_rotate) & (df['Trial_num'] <=3) & 
                 (df['Gesture'] != '1') & (df['Gesture'] != '0') & (df['Gesture'] != '3') & 
                 (df['Gesture'] != '10') & (df['Gesture'] != '11')]
    
    if flipped == True:
        print('Flipping test data')
        df_test_temp = flipping_sensor(df_test.iloc[:, :8])
    else:
        df_test_temp = df_test.iloc[:, :8].copy()
    
    df_test_calibrated = pd.concat((pd.concat((df_test_temp, df_test_temp), axis = 1).iloc[:,  0 + shift:8 + shift], df_test.iloc[:, 8:]), axis = 1)
    
    df_test_calibrated = pd.DataFrame(df_test_calibrated.values, columns = df_test.columns)
            
    return df_train, df_test, df_test_calibrated

In [5]:
def flipping_sensor(df_unflipped):
    A = df_unflipped.iloc[:, :8]
    A = A.loc[:, ::-1].values
    
    df_flipped = df_unflipped.copy()
    df_flipped.iloc[:, :8] = A
    
    return df_flipped

# Use correlation to align the sensor
### Known fact:

Subject 1008, 0100 and 9998 worn the sensor upside down between sessions

In [22]:
%%time
stats = []
subject_list = list(df['ID'].unique())


### for those who wore the sensor upside down between 2 sessions
# for subject in ['0001', '0009', '0010', '0010', '0011', '0012', '0100', '1001', '1003', '1004', '1005', '1007', '1111', '1234', '9999']: 

### for those who wore the sensor upside down between 2 sessions
for subject in ['0100', '1008', '9998']: 
    session_list = ['S1', 'S2']
    
    for session_test in session_list:
        session_train = [x for x in session_list if x != session_test]
        session_reference = session_train[0]
        
        df_train_calibrated_all = df[(df['ID'] == subject) & 
                      (df['session'] == session_reference) & (df['Trial_num'] <=3) & 
                      (df['Gesture'] != '1') & (df['Gesture'] != '0') & (df['Gesture'] != '3') & 
                      (df['Gesture'] != '10') & (df['Gesture'] != '11')]
        print('===================================================')
        print(f'Rotation alignment for training data for {subject}')
        print('===================================================')
        
        for session_train_to_rotate in session_train[1:]:
            if len(session_train[1:]) == 0:
                flipped_train = False
                shift_train_temp = 0 
                continue
            print(f'Session to reference: {session_reference}')
            print(f'Session to rotate: {session_train_to_rotate}')
            flipped_train, shift_train = CC_to_shift(df, 
                                                    subject_reference = subject,
                                                    subject_to_rotate = subject,
                                                    calibration_gesture_list = ['8', '5', '4'], 
                                                    session_reference = session_reference, 
                                                    session_to_rotate = session_train_to_rotate) 
        
            
            _ , _, df_train_calibrated = shift_to_rotation(df, shift_train, 
                                                           subject_reference = subject,
                                                           subject_to_rotate = subject,
                                                           session_reference = session_reference, 
                                                           session_to_rotate = session_train_to_rotate, 
                                                           flipped = flipped_train)
            
            df_train_calibrated_all = pd.concat((df_train_calibrated_all, df_train_calibrated), axis = 0)
            print(f'Session {session_train_to_rotate}, Flipping needed? {flipped_train}, shift {shift_train}')
            
        print('===================================================')
        print(f'Rotation alignment for test data for {subject}')
        print('===================================================')
        print(f'Session to reference: {session_reference}')
        print(f'Session to rotate: {session_test}')
        flipped_test, shift_test = CC_to_shift(df, 
                                               subject_reference = subject, 
                                               subject_to_rotate = subject, 
                                               calibration_gesture_list = ['8', '5', '4'], 
                                               session_reference = session_reference, 
                                               session_to_rotate = session_test)    
            
        _, df_test, df_test_calibrated = shift_to_rotation(df, shift_test, 
                                                           subject_reference = subject, 
                                                           subject_to_rotate = subject, 
                                                           session_reference = session_reference, 
                                                           session_to_rotate = session_test, 
                                                           flipped = flipped_test)
        print(f'Session {session_test}, Flipping needed? {flipped_test}, shift {shift_test}')
        if len(df_test_calibrated) == 0:
            continue
            
        df_train = df_train_calibrated_all[df_train_calibrated_all['Gesture'] != '8'].copy()
        df_test = df_test[df_test['Gesture'] != '8'].copy()
        
        if flipped_test == False and shift_test == 0:
            df_test_calibrated = df_test
        else:
            df_test_calibrated = df_test_calibrated[df_test_calibrated['Gesture'] != '8'].copy()

        df_train.reset_index(inplace = True, drop = True)
        df_val = df_train.groupby(['Gesture', 'session', 'Trial_num']).tail(100)

        df_train.loc[df_val.index, :] = np.nan
        df_train.dropna(inplace = True)
        
        # Data preprocessing for training data, which includes .... 
        # (1) Spliiting the raw data into windows, 
        # (2) Label encoding the target variables and 
        # (3) Standardization for the training data
        
        print(f'Data preprocessing and filtering......')
        X_train, y_train, _= preprocess(df_train, window_size = 52, nonoverlap_size = 5, highpass = True)
        X_val, y_val, _= preprocess(df_val, window_size = 52, nonoverlap_size = 5, highpass = True)

        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_val = le.transform(y_val)   

        y_train = to_categorical(y_train)
        y_val = to_categorical(y_val)

        standard_scaler = StandardScaler()
        X_train = standard_scaler.fit_transform(X_train.reshape(-1, 8)).reshape(X_train.shape)
        X_val = standard_scaler.transform(X_val.reshape(-1, 8)).reshape(X_val.shape)

        X_train = X_train.reshape((-1, X_train.shape[1], X_train.shape[2], 1))
        X_val = X_val.reshape((-1, X_val.shape[1], X_val.shape[2], 1))
        
        # cross validate 3 trials and do the same data preprocessing as for training data
        for trial_num in [1, 2, 3]:
            
            print(f'\nCurrent test trial {trial_num}')
            X_test, y_test, _= preprocess(df_test[df_test['Trial_num'] == trial_num], window_size = 52, nonoverlap_size = 5, highpass = True)
            X_test_calibrated, y_test_calibrated, _ = preprocess(df_test_calibrated[df_test_calibrated['Trial_num'] == trial_num], window_size = 52, nonoverlap_size = 5)

            y_test = le.transform(y_test)
            y_test_calibrated = le.transform(y_test_calibrated)

            y_test = to_categorical(y_test)
            y_test_calibrated = to_categorical(y_test_calibrated)

            X_test = standard_scaler.transform(X_test.reshape(-1, 8)).reshape(X_test.shape)
            X_test_calibrated = standard_scaler.transform(X_test_calibrated.reshape(-1, 8)).reshape(X_test_calibrated.shape)

            X_test = X_test.reshape((-1, X_test.shape[1], X_test.shape[2], 1))
            X_test_calibrated = X_test_calibrated.reshape((-1, X_test_calibrated.shape[1], X_test_calibrated.shape[2], 1))
            
            # Setting the parameters for the CNN model
            model = get_CNN_model(X_train.shape, y_train.shape[1], dr = 0.5, wd = 0.01)

            model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), 
                        loss = tf.keras.losses.CategoricalCrossentropy(),
                        metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'acc')])
            reduce_lr = ReduceLROnPlateau(monitor = 'val_acc', patience = 3, mode = 'max', verbose = 1)
            early_stopping = EarlyStopping(monitor = 'val_acc', patience = 10, mode = 'max', verbose = 1)

            model_checkpoint = ModelCheckpoint(filepath = subject + 'Rotation_' + session_test +'_save_best_only/checkpoint_{val_acc:.2f}',
                                            monitor = 'val_acc', 
                                            save_weights_only = False, 
                                            save_best_only = True)
            
            # Looks for the trained model, if there is none, then start training
            if os.path.exists(subject + 'Rotation_' + session_test + '_save_best_only/') == False:
                print('No previously trained model found')
                history = model.fit(X_train, y_train, 
                                  validation_data = (X_val, y_val), 
                                  callbacks = [reduce_lr, early_stopping, model_checkpoint], 
                                  epochs = 30, verbose = 0)
            else:
                print('Found previously trained model')
                
            # Load the pretrained model on that subject
            lastmodel = glob2.glob(subject + 'Rotation_' + session_test + '_save_best_only/*')[-1]
            model_load = load_model(lastmodel)
            
            
            # Get the accuracy before armband calibration
            y_pred = np.argmax(model_load.predict(X_test), axis = -1)
            y_true = np.argmax(y_test, axis = -1)        
            acc = balanced_accuracy_score(y_true, y_pred)

            matrix_before = confusion_matrix(y_true, y_pred)
            cm_before = pd.DataFrame(matrix_before, columns = Gesture_list, index = Gesture_list)
            cm_before['ID'] = subject
            cm_before['session'] = session_test
            
            print(f'Accuracy before {acc:.3f}')

            y_pred_calibrated = np.argmax(model_load.predict(X_test_calibrated), axis = -1)
            y_true_calibrated = np.argmax(y_test_calibrated, axis = -1)  

            # Get the accuracy after armband calibration
            acc_calibrated = balanced_accuracy_score(y_true_calibrated, y_pred_calibrated)
            matrix_after = confusion_matrix(y_true_calibrated, y_pred_calibrated)
            cm_after = pd.DataFrame(matrix_after, columns = Gesture_list, index = Gesture_list)
            cm_after['ID'] = subject
            cm_after['session'] = session_test        
            print(f'Accuracy after {acc_calibrated:.3f}')
            stats.append({'ID': subject,
                          'session': session_test,
                          'Trial_num': trial_num,
                          'Accuracy_before_armband_rotation': acc,
                          'Accuracy_after_armband_rotation':acc_calibrated,
                          'confusion_matrix': cm_before,
                          'confusion_matrix_calibrated': cm_after})

Rotation alignment for training data for 0100
Rotation alignment for test data for 0100
Session to reference: S2
Session to rotate: S1
Flipping test data
Session S1, Flipping needed? True, shift 1
Data preprocessing and filtering......

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9

Current test trial 1

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9
Found previously trained model
Accuracy before 0.196
Accuracy after 0.812

Current test trial 2

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9
Found previously trained model
Accuracy before 0.182
Accuracy after 0.768

Current test trial 3

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 

In [23]:
stats = pd.DataFrame(stats).set_index('ID')
stats.to_pickle('Between_session_Rotation_0817_both.pkl')

# Data Augmentation

Make copies of all potential shifts, both in non-flipped and flipped version. Since we have 8 different sensor position in both orientation, so we have 16 different datasets to train with. Reference: https://ieeexplore.ieee.org/document/9127387

In [10]:
def data_augmentation(df, shift_list):
    df_all = df.copy()
    df_flipped = flipping_sensor(df_all)
    df_flipped['Trial_num'] = df_flipped['Trial_num'] + 3
    df_all = pd.concat((df_all, df_flipped), axis = 0)
    count = 2
    for flipped in [False, True]:
        if flipped == True:
            df = flipping_sensor(df)
        for shift in shift_list:
            df_temp = df.iloc[:, :8]
            df_shift = pd.concat((df_temp, df_temp, df_temp), axis = 1).iloc[:, 8 + shift: 16 + shift].values
            df_shift = pd.DataFrame(df_shift, columns = df_temp.columns, index = df.index)
            df_shift = pd.concat((df_shift, df.iloc[:, 8:]), axis = 1)
            df_shift['Trial_num'] = df_shift['Trial_num'] + 3*count
            count +=1
    #         fig, ax = plt.subplots(2)
    #         df_temp.mean().plot(kind = 'bar', ax = ax[0])
    #         df_shift.iloc[:, :8].mean().plot(kind = 'bar', ax = ax[1])
            df_all = pd.concat((df_all, df_shift), axis = 0)
        
    return df_all

In [20]:
%%time
stats = []

### for those who wore the sensor upside down between 2 sessions
# for subject in ['0001', '0009', '0010', '0010', '0011', '0012', '0100', '1001', '1003', '1004', '1005', '1007', '1111', '1234', '9999']: 

### for those who wore the sensor upside down between 2 sessions
for subject in ['0100', '1008', '9998']: 
    session_list = ['S1', 'S2']
            
    for session_test in session_list:
        session_train = [x for x in session_list if x != session_test]
        session_reference = session_train[0]
        
        df_train_calibrated_all = df[(df['ID'] == subject) & 
                      (df['session'] == session_reference) & (df['Trial_num'] <=3) & 
                      (df['Gesture'] != '1') & (df['Gesture'] != '0') & (df['Gesture'] != '3') & 
                      (df['Gesture'] != '10') & (df['Gesture'] != '11')]
        
        
        length_before = len(df[(df['ID'] == subject) & 
                               (df['session'] != session_test) &
                               (df['Trial_num'] <=3) & 
                               (df['Gesture'] != '1') & (df['Gesture'] != '0') & (df['Gesture'] != '3') & 
                               (df['Gesture'] != '10') & (df['Gesture'] != '11')])
        print('\n===========================================')
        print(f'Data augmentation for training data.....')
        print('===========================================')
        
        df_train_calibrated_all = data_augmentation(df_train_calibrated_all, shift_list = [-3, -2, -1, 1, 2, 3, 4])
        
        for session_train_to_augment in session_train[1:]:

            df_train_calibrated = df[(df['ID'] == subject) & 
                      (df['session'] == session_train_to_augment) & (df['Trial_num'] <=3) & 
                      (df['Gesture'] != '1') & (df['Gesture'] != '0') & (df['Gesture'] != '3') & 
                      (df['Gesture'] != '10') & (df['Gesture'] != '11')]
            
            df_train_calibrated = data_augmentation(df_train_calibrated, shift_list = [-3, -2, -1, 1, 2, 3, 4])
            
            df_train_calibrated_all = pd.concat((df_train_calibrated_all, df_train_calibrated), axis = 0)
            
        length_after = len(df_train_calibrated_all) 
        print(f'Length before data augmentation {length_before}')
        print(f'Length after data augmentation {length_after}')
        
        print('\n===========================================')
        print(f'Read the test data......................')
        print('===========================================') 
        
        df_test = df[(df['ID'] == subject) & 
                      (df['session'] == session_test) & (df['Trial_num'] <=3) & 
                      (df['Gesture'] != '1') & (df['Gesture'] != '0') & (df['Gesture'] != '3') & 
                      (df['Gesture'] != '10') & (df['Gesture'] != '11')]
            
        df_train = df_train_calibrated_all[df_train_calibrated_all['Gesture'] != '8'].copy()
        df_test = df_test[df_test['Gesture'] != '8'].copy()

        df_train.reset_index(inplace = True, drop = True)
        df_val = df_train.groupby(['Gesture', 'session', 'Trial_num']).tail(100)

        df_train.loc[df_val.index, :] = np.nan
        df_train.dropna(inplace = True)
        print(f'Data preprocessing and filtering\n')
        X_train, y_train, _= preprocess(df_train, window_size = 52, nonoverlap_size = 5, highpass = True)
        X_val, y_val, _= preprocess(df_val, window_size = 52, nonoverlap_size = 5, highpass = True)

        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_val = le.transform(y_val)   

        y_train = to_categorical(y_train)
        y_val = to_categorical(y_val)

        standard_scaler = StandardScaler()
        X_train = standard_scaler.fit_transform(X_train.reshape(-1, 8)).reshape(X_train.shape)
        X_val = standard_scaler.transform(X_val.reshape(-1, 8)).reshape(X_val.shape)

        X_train = X_train.reshape((-1, X_train.shape[1], X_train.shape[2], 1))
        X_val = X_val.reshape((-1, X_val.shape[1], X_val.shape[2], 1))
        ########################## Add code to cross validate 3 trials 

        for trial_num in [1, 2, 3]:
            X_test, y_test, _= preprocess(df_test[df_test['Trial_num'] == trial_num], window_size = 52, nonoverlap_size = 5, highpass = True)


            y_test = le.transform(y_test)
            y_test = to_categorical(y_test)

            X_test = standard_scaler.transform(X_test.reshape(-1, 8)).reshape(X_test.shape)
            X_test = X_test.reshape((-1, X_test.shape[1], X_test.shape[2], 1))
            

            model = get_CNN_model(X_train.shape, y_train.shape[1], dr = 0.5, wd = 0.01)

            model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), 
                        loss = tf.keras.losses.CategoricalCrossentropy(),
                        metrics = [tf.keras.metrics.CategoricalAccuracy(name = 'acc')])
            reduce_lr = ReduceLROnPlateau(monitor = 'val_acc', patience = 3, mode = 'max', verbose = 0)
            early_stopping = EarlyStopping(monitor = 'val_acc', patience = 10, mode = 'max', verbose = 0)

            model_checkpoint = ModelCheckpoint(filepath = subject + 'DataAugmentation_' + session_test +'_save_best_only/checkpoint_{val_acc:.2f}',
                                            monitor = 'val_acc', 
                                            save_weights_only = False, 
                                            save_best_only = True, verbose = 1)
            if os.path.exists(subject + 'DataAugmentation_' + session_test + '_save_best_only/') == False:
                print('No previously trained model found')
                history = model.fit(X_train, y_train, 
                                  validation_data = (X_val, y_val), 
                                  callbacks = [reduce_lr, early_stopping, model_checkpoint], 
                                  epochs = 30, verbose = 0)
            else:
                print('Found previously trained model')

            lastmodel = glob2.glob(subject + 'DataAugmentation_' + session_test + '_save_best_only/*')[-1]
            model_load = load_model(lastmodel)


            # Get the accuracy before armband calibration
            y_pred = np.argmax(model_load.predict(X_test), axis = -1)
            y_true = np.argmax(y_test, axis = -1)        
            acc = balanced_accuracy_score(y_true, y_pred)

            matrix_before = confusion_matrix(y_true, y_pred)
            cm_before = pd.DataFrame(matrix_before, columns = Gesture_list, index = Gesture_list)
            cm_before['ID'] = subject
            cm_before['session'] = session_test
            print(f'Accuracy: {acc:.3f}')

            stats.append({'ID': subject,
                          'session': session_test,
                          'Trial_num': trial_num,
                          'Accuracy_with_data_augmentation': acc,
                          'confusion_matrix': cm_before})


Data augmentation for training data.....
Length before data augmentation 20974
Length after data augmentation 335584

Read the test data......................
Data preprocessing and filtering


Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9

Gesture: 2

Gesture: 4

Gesture: 5

Gesture: 6

Gesture: 7

Gesture: 9
No previously trained model found

Epoch 00001: val_acc improved from -inf to 0.91562, saving model to 0100DataAugmentation_S1_save_best_only\checkpoint_0.92
INFO:tensorflow:Assets written to: 0100DataAugmentation_S1_save_best_only\checkpoint_0.92\assets

Epoch 00002: val_acc did not improve from 0.91562

Epoch 00003: val_acc improved from 0.91562 to 0.92778, saving model to 0100DataAugmentation_S1_save_best_only\checkpoint_0.93
INFO:tensorflow:Assets written to: 0100DataAugmentation_S1_save_best_only\checkpoint_0.93\assets

Epoch 00004: val_acc improved from 0.92778 to 0.97431, savi


Epoch 00005: val_acc did not improve from 0.74931

Epoch 00006: val_acc improved from 0.74931 to 0.89410, saving model to 1008DataAugmentation_S1_save_best_only\checkpoint_0.89
INFO:tensorflow:Assets written to: 1008DataAugmentation_S1_save_best_only\checkpoint_0.89\assets

Epoch 00007: val_acc improved from 0.89410 to 0.90243, saving model to 1008DataAugmentation_S1_save_best_only\checkpoint_0.90
INFO:tensorflow:Assets written to: 1008DataAugmentation_S1_save_best_only\checkpoint_0.90\assets

Epoch 00008: val_acc did not improve from 0.90243

Epoch 00009: val_acc improved from 0.90243 to 0.90451, saving model to 1008DataAugmentation_S1_save_best_only\checkpoint_0.90
INFO:tensorflow:Assets written to: 1008DataAugmentation_S1_save_best_only\checkpoint_0.90\assets

Epoch 00010: val_acc did not improve from 0.90451

Epoch 00011: val_acc did not improve from 0.90451

Epoch 00012: val_acc did not improve from 0.90451

Epoch 00013: val_acc did not improve from 0.90451

Epoch 00014: val_acc 


Epoch 00006: val_acc improved from 0.84653 to 0.90035, saving model to 9998DataAugmentation_S2_save_best_only\checkpoint_0.90
INFO:tensorflow:Assets written to: 9998DataAugmentation_S2_save_best_only\checkpoint_0.90\assets

Epoch 00007: val_acc improved from 0.90035 to 0.91528, saving model to 9998DataAugmentation_S2_save_best_only\checkpoint_0.92
INFO:tensorflow:Assets written to: 9998DataAugmentation_S2_save_best_only\checkpoint_0.92\assets

Epoch 00008: val_acc did not improve from 0.91528

Epoch 00009: val_acc did not improve from 0.91528

Epoch 00010: val_acc did not improve from 0.91528

Epoch 00011: val_acc did not improve from 0.91528

Epoch 00012: val_acc did not improve from 0.91528

Epoch 00013: val_acc did not improve from 0.91528

Epoch 00014: val_acc did not improve from 0.91528

Epoch 00015: val_acc did not improve from 0.91528

Epoch 00016: val_acc did not improve from 0.91528

Epoch 00017: val_acc did not improve from 0.91528
Accuracy: 0.880

Gesture: 2

Gesture: 4

G

In [21]:
stats = pd.DataFrame(stats).set_index('ID')
# stats.to_pickle('Between_session_DataAugmentation_all_shifts_0817_both.pkl')